In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('..\processed_data\stock_price_monthly_2003_2024_processed.csv')
df["date"] = pd.to_datetime(df["date"]).dt.to_period('M')
df.head(10)

C:\Users\vietx\AppData\Local\Temp\ipykernel_24028\2074313005.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["date"] = pd.to_datetime(df["date"]).dt.to_period('M')


,date,NCUSIP,TICKER,COMNAM,PRC,VOL,RET,SHROUT,NasdaqDummy
0,2002-01,29274A10,EWST,ENERGY WEST INC,11.3000,474.0,-0.013100,2566.0,1
1,2002-02,29274A10,EWST,ENERGY WEST INC,10.7000,495.0,-0.053097,2569.0,1
2,2002-03,29274A10,EWST,ENERGY WEST INC,10.4000,908.0,-0.015888,2571.0,1
3,2002-04,29274A10,EWST,ENERGY WEST INC,9.9500,1346.0,-0.043269,2571.0,1
4,2002-05,29274A10,EWST,ENERGY WEST INC,10.0975,647.0,0.014824,2571.0,1
5,2002-06,29274A10,EWST,ENERGY WEST INC,9.7200,778.0,-0.024016,2574.0,1
6,2002-07,29274A10,EWST,ENERGY WEST INC,8.7500,576.0,-0.099794,2574.0,1
7,2002-08,29274A10,EWST,ENERGY WEST INC,8.9000,577.0,0.017143,2574.0,1
8,2002-09,29274A10,EWST,ENERGY WEST INC,8.7500,514.0,-0.001685,2576.0,1
9,2002-10,29274A10,EWST,ENERGY WEST INC,8.5800,431.0,-0.019429,2586.0,1


In [2]:
 # Sort data by NCUSIP (Stock ID) and date
df = df.sort_values(by=["NCUSIP", "TICKER", "date"])


df["RET"] = pd.to_numeric(df["RET"], errors="coerce")

# remove rows where RET = -66.0, -77.0, -88.0, -99.0
df = df[(df["RET"] != -66.0) & (df["RET"] != -77.0) & (df["RET"] != -88.0) & (df["RET"] != -99.0)]

# Compute past 12-month return (momentum measure)
df["past_return"] = (
    df.groupby("NCUSIP")["RET"]
    .rolling(window=12, min_periods=12)
    .apply(lambda x: np.prod(1 + x) - 1, raw=True)
    .reset_index(level=0, drop=True)
)

# Compute past 12-month return while skipping the most recent month
df["past_return_skip"] = (
    df.groupby("NCUSIP")["RET"]
    .rolling(window=12, min_periods=12)
    .apply(lambda x: np.prod(1 + x[:-1]) - 1, raw=True)
    .reset_index(level=0, drop=True)
)

# Compute past 12-month variance
df["past_variance"] = (
    df.groupby("NCUSIP")["RET"]
    .rolling(window=12, min_periods=12)
    .var()
    .reset_index(level=0, drop=True)
)

# Rk is the rate of return of a firm lagged k years for k 5 0,1,2,3,4
df["R1"] = df.groupby("NCUSIP")["RET"].shift(12)
df["R2"] = df.groupby("NCUSIP")["RET"].shift(24)
df["R3"] = df.groupby("NCUSIP")["RET"].shift(36)
df["R4"] = df.groupby("NCUSIP")["RET"].shift(48)


df

,date,NCUSIP,TICKER,COMNAM,PRC,VOL,RET,SHROUT,NasdaqDummy,past_return,past_return_skip,past_variance,R1,R2,R3,R4
141855,2014-10,00030710,AAC,A A C HOLDINGS INC,21.7800,54319.0,NaN,20673.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141856,2014-11,00030710,AAC,A A C HOLDINGS INC,28.8600,22234.0,0.325069,21081.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141857,2014-12,00030710,AAC,A A C HOLDINGS INC,30.9200,21454.0,0.071379,21081.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141858,2015-01,00030710,AAC,A A C HOLDINGS INC,25.5200,12430.0,-0.174644,21081.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141859,2015-02,00030710,AAC,A A C HOLDINGS INC,35.9900,13622.0,0.410266,21081.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94083,2017-09,U7260311,PLPM,PLANET PAYMENT INC,4.2900,57108.0,0.083333,49971.0,1,0.156333,0.067385,0.010623,0.033426,-0.043919,-0.186722,0.040984
94084,2017-10,U7260311,PLPM,PLANET PAYMENT INC,4.4700,293128.0,0.041958,50012.0,1,0.280801,0.229225,0.010074,-0.059299,0.060071,-0.137755,0.062992
94085,2017-11,U7260311,PLPM,PLANET PAYMENT INC,4.5000,104056.0,0.006711,50012.0,1,0.119402,0.111939,0.008491,0.151862,-0.050000,-0.026509,-0.129630
312944,2024-02,U9154A20,UMAC,UNUSUAL MACHINES INC,2.8400,11929.0,NaN,8717.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df[df["NCUSIP"] == "U7260311"].sort_values(by="date")

,date,NCUSIP,TICKER,COMNAM,PRC,VOL,RET,SHROUT,NasdaqDummy,past_return,past_return_skip,past_variance,R1,R2,R3,R4
94026,2012-12,U7260311,PLPM,PLANET PAYMENT INC,3.4800,2469.0,NaN,53659.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94027,2013-01,U7260311,PLPM,PLANET PAYMENT INC,2.8500,4676.0,-0.181035,53659.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94028,2013-02,U7260311,PLPM,PLANET PAYMENT INC,3.0100,2295.0,0.056140,53701.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94029,2013-03,U7260311,PLPM,PLANET PAYMENT INC,2.9400,7512.0,-0.023256,53707.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94030,2013-04,U7260311,PLPM,PLANET PAYMENT INC,3.1700,12200.0,0.078231,53719.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94031,2013-05,U7260311,PLPM,PLANET PAYMENT INC,2.9800,19435.0,-0.059937,53719.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94032,2013-06,U7260311,PLPM,PLANET PAYMENT INC,2.7600,88065.0,-0.073826,54483.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94033,2013-07,U7260311,PLPM,PLANET PAYMENT INC,3.0100,55033.0,0.090580,54568.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94034,2013-08,U7260311,PLPM,PLANET PAYMENT INC,2.4400,22783.0,-0.189369,54568.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94035,2013-09,U7260311,PLPM,PLANET PAYMENT INC,2.5400,9545.0,0.040984,54571.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# get year = 2013
_test = df[(df["NCUSIP"] == "U7260311") & (df["date"].dt.year == 2016)]

mult = 1
for _t in _test['RET']:
    mult = mult * (1 + _t)
mult - 1

0.3377013626824048

In [5]:
df["ME"] = abs(df["PRC"]) * df["SHROUT"]
# turnover
df["T-O"] = df["VOL"] / df["SHROUT"]

In [6]:
df.drop(columns=["VOL", "vwretd"], inplace=True, errors="ignore")
df.sort_values(by=["date", "NCUSIP"], inplace=True)
df

,date,NCUSIP,TICKER,COMNAM,PRC,RET,SHROUT,NasdaqDummy,past_return,past_return_skip,past_variance,R1,R2,R3,R4,ME,T-O
534646,2002-01,00036020,AAON,AAON INC,19.6995,-0.194953,8666.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,170715.867,0.618278
416442,2002-01,00036110,AIR,A A R CORP,8.6000,-0.042730,26859.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230987.400,0.669198
637054,2002-01,00040010,ABCB,A B C BANCORP,13.5000,0.011236,9999.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134986.500,0.594859
836648,2002-01,00078810,ABWG,A B WATLEY GROUP INC,1.4400,-0.389830,12887.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18557.280,0.688523
776558,2002-01,00086T10,ACMR,AC MOORE ARTS AND CRAFTS INC,27.3800,-0.071864,7466.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,204419.080,1.007233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188697,2024-12,98985Y10,ZYME,ZYMEWORKS INC DE,14.6400,0.039035,69163.0,1,0.409047,0.356111,0.011967,0.171364,0.018135,NaN,NaN,1012546.320,1.530081
223468,2024-12,98986M10,ZYXI,ZYNEX INC,8.0100,-0.037260,31846.0,1,-0.264463,-0.235996,0.007309,0.188865,0.013848,-0.221094,-0.035817,255086.460,0.506060
295066,2024-12,98987D30,ZVSA,ZYVERSA THERAPEUTICS INC,1.0600,-0.012944,2344.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2484.640,14.294369
251591,2024-12,G6516310,JOBY,JOBY AVIATION INC,8.1300,-0.091620,765086.0,0,0.222556,0.345864,0.075974,0.115772,-0.186893,-0.079445,NaN,6220149.180,5.434619


In [7]:
# save to csv
df.to_csv('..\\models_data\\stock_price_monthly_2003_2024.csv', index=False)